In [1]:
import pandas as pd
df= pd.read_csv(r"C:\Users\indra\Desktop\DSG PROJECT\BH25\Training_Data\train.csv") # the train csv is loaded

In [2]:
df["power"].unique() # checked the unique powers available of which many are noises

array(['rot', 'grin', 'gelb', 'lila', 'blau', 'grun', 'trun', 'gela',
       'brun'], dtype=object)

In [3]:
data = {
    'power': ['rot', 'grin', 'geld', 'lila', 'blau', 'btau', 'rot', 'helb',
               'rod', 'grin', 'got', 'not', 'lila', 'trun', 'gela', 'gelt',
               'baau', 'liga', 'gelb', 'plau', 'brun', 'rot', 'lili', 'slau',
               'lisa', 'blru', 'grus', 'lula', 'gele', 'gaun']
}

# Count the occurrences of each unique element
value_counts = df['power'].value_counts()

value_counts # this was done to identify the outliers or noise


power
rot     2062
blau    2022
lila    2009
gelb    1728
grun    1390
grin     199
trun     199
brun     199
gela     192
Name: count, dtype: int64

In [4]:
df = df[df['power'].isin(['rot', 'blau', 'lila', 'grun', 'gelb'])] # the rows with noise were droped and only 5 types of powers are kept for training

In [5]:
df

,video_id,element,motion,power,speed,video_summary
0,1,Erde,linear,rot,9.6,"(3.8147, 31.94809)"
2,3,Feuer,random,gelb,9.6,"(3.8147, 8.58307)"
3,4,Erde,zigzag,lila,6.9,"(-24.79553, -0.95367)"
4,5,Erde,linear,lila,9.6,"(7.62939, 22.88818)"
5,6,Erde,circular,blau,9.6,"(4.76837, -26.70288)"
...,...,...,...,...,...,...
9995,9996,Feuer,zigzag,rot,9.6,"(-1.90735, -11.44409)"
9996,9997,Wasser,shm,gelb,9.6,"(0.0, 13.14282)"
9997,9998,Feuer,circular,lila,6.9,"(17.64297, 10.49042)"
9998,9999,Boden,zigzag,gelb,9.6,"(-21.45767, 11.44409)"


In [6]:
import cv2
import pandas as pd


def extract_brightest_color(video_path):
    cap = cv2.VideoCapture(video_path) # cv is to read the frames of the videos
    success, frame = cap.read()  # Reads the frames

    if success: # checks if frames are available
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # converts the frame to rgb 
        brightest_pixel = frame_rgb.reshape(-1, 3).max(axis=0)  # gets RGB values of the brightest pixel
        return [int(value) for value in brightest_pixel]  
    else:
        print(f"Could not read video: {video_path}")  # notifies if the video is unaccessible 
        return None
    cap.release()

def update_csv_with_brightest_color(df, video_folder):
    brightest_colors = []

    for video_id in df['video_id']:
        video_path = f"{video_folder}/{video_id}.mp4"  # reads the videos with the ids in the updated df
        rgb = extract_brightest_color(video_path) # runs the function to extract the brightest colour of the frame
        brightest_colors.append(rgb)

    df['shape_rgb_values'] = brightest_colors # creates another column in the df and stores the value of brightest colours
    return df

video_folder = "./Train_Videos"  # folder with the videos


df= update_csv_with_brightest_color(df, video_folder)
print("done")


done


In [7]:
df[['R', 'G', 'B']] = pd.DataFrame(df['shape_rgb_values'].tolist(), index=df.index) # the tuple is seperated to different columns as R G B
df

,video_id,element,motion,power,speed,video_summary,shape_rgb_values,R,G,B
0,1,Erde,linear,rot,9.6,"(3.8147, 31.94809)","[103, 80, 248]",103,80,248
2,3,Feuer,random,gelb,9.6,"(3.8147, 8.58307)","[129, 212, 255]",129,212,255
3,4,Erde,zigzag,lila,6.9,"(-24.79553, -0.95367)","[123, 54, 99]",123,54,99
4,5,Erde,linear,lila,9.6,"(7.62939, 22.88818)","[121, 44, 98]",121,44,98
5,6,Erde,circular,blau,9.6,"(4.76837, -26.70288)","[195, 255, 180]",195,255,180
...,...,...,...,...,...,...,...,...,...,...
9995,9996,Feuer,zigzag,rot,9.6,"(-1.90735, -11.44409)","[101, 72, 240]",101,72,240
9996,9997,Wasser,shm,gelb,9.6,"(0.0, 13.14282)","[129, 218, 255]",129,218,255
9997,9998,Feuer,circular,lila,6.9,"(17.64297, 10.49042)","[121, 47, 97]",121,47,97
9998,9999,Boden,zigzag,gelb,9.6,"(-21.45767, 11.44409)","[136, 216, 255]",136,216,255


# power: LogisticRegression

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
le = LabelEncoder()
df["power"] = le.fit_transform(df["power"])
X = df[['R', 'G', 'B']]  # defines the X value of the model
y = df["power"]             # defines the y value of the model


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Split data into training and test sets 

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_scaled = scaler.fit_transform(X)
# the values are preprocessed with standardscalar before training with Logistic regression for more accuracy


In [9]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

model.fit(X_train_scaled, y_train)

model.score(X_test_scaled, y_test)



1.0

In [10]:
import joblib
joblib.dump(model, './power_model.pkl')
joblib.dump(le, 'label_encoder_power.pkl')
# the model and the label encodes are saved

['label_encoder_power.pkl']